In [639]:
%%time
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
import tools.Sample_Tools as smpl
import Analysis_Funs as af
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import talib

from tools.Test_Tools import FactorTest
import tools.Pretreat_Tools as pretreat
from base.JuUnits import excute_for_multidates

import toys
# %load_ext autoreload
# %autoreload 2
# %aimport tools.Test_Tools,toys




# data = smpl.get_data(['000002'],end='2020-11-29',gap=528).data

# ttt =TTT(data)
# ttt.fit()

# fig = plt.figure(figsize=(1620/72,620/72))
# ax1= fig.add_subplot(2,1,1)
# # pd.concat([data.close,ht.ind_df['zhouqi']*5,ht.ind_df['jieduan']],axis=1).plot(secondary_y=['zhouqi','jieduan'],ax=ax1)
# # pd.concat([data.close,ht.ind_df['inphase'],ht.ind_df['quadrature']],axis=1).plot(secondary_y=['inphase','quadrature'],ax=ax1)
# pd.concat([data.close,ttt.ind_df['TTT']],axis=1).plot(secondary_y=['TTT'],ax=ax1)
# plt.grid()
# ax2= fig.add_subplot(6,1,4,sharex=ax1)
# data.volume.plot(kind='bar',ax=ax2)
# ax2.tick_params(labelbottom=True)  #  解决share后标签不可见的问题。
# ax2.xaxis.set_major_locator(ticker.MaxNLocator(6))



Wall time: 0 ns


In [642]:


# data = smpl.get_sample_by_zs(name='上证50', end='2020-11-28', gap=150,  only_main=True)
# returns = smpl.get_current_return(data,'close')
# returns.name = 'returns'
# ret_forward = smpl.get_forward_return(data,'close')
# ret_forward.name = 'ret_forward'
# # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
# data = pd.concat([data.data, returns, ret_forward], axis=1)
# data = data.assign(vwap=data.amount/(data.volume*100))
# data = smpl.add_marketvalue_industry(data)
# data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)
# data
# ind_added[['totalCapital','industry']].sort_index()
pretreat.neutralize(data.close, data['ind'],categorical=['ind'])
adv20_ind = sma(volume, 20)
close_ind
vwap_ind
low_ind
co_mixed_ind = ((close * 0.60733) + (Open * (1 - 0.60733)))
oh_mixed_ind = ((Open * 0.868128) + (high * (1 - 0.868128)))
volume_ind
adv40_ind = sma(volume, 40).fillna(value=0)
lv_mixed_ind = IndNeutralize(((low * 0.721001) + (vwap * (1 - 0.721001))), ind)

date        code  
2020-04-17  600000   -11.1750
            600028     0.2475
            600030    -1.9950
            600031    -2.7750
            600036    11.4850
                       ...   
2020-11-26  601995    36.8250
            603259    -1.0625
            603288    15.8625
            603501    -1.7300
            603986    -2.7400
Name: close, Length: 7165, dtype: float64

In [635]:
%%time

def check_quality(ind_binned_ret):
    x = ind_binned_ret
    # x = x/np.abs(x).max()+0.000001
    r_cor = np.corrcoef(x.rank(ascending=False),x.index)[0][1]
    # print(r_cor,x.idxmin(),x.idxmax())
    
    if np.abs(r_cor)>0.45:
        if x.idxmax() in [1,2]:
            return 'valid_left'
        if x.idxmax() in [9,10]:
            return 'valid_right'
        return 'invalid'
    
    if np.abs(r_cor) < 0.16:
        if x.idxmax() in [5,6]:
            return 'sem_mid'
        if x.idxmax() in [1,10]:
            x[x.idxmax()]=0
            if x.idxmax() in [1,10]:
                return 'sem_bilateral'
        return 'invalid'
    
    return 'invalid'

# fun_name = 'alpha101'
for idx in range(1,102):
    fun_name = 'alpha'+str(idx)
    print(fun_name)
    
    params = inspect.signature(getattr(toys, fun_name)).parameters.keys()
    if 'ind' in params:
        # raise 'pass' # 暂时先不处理中性化问题。
        indx = excute_for_multidates(data, lambda x: getattr(toys, fun_name)(*[x[param].copy() for param in params]) ,level=1)
    else:
        indx = excute_for_multidates(data, lambda x: getattr(toys, fun_name)(*[x[param].copy() for param in params]) ,level=1)
    indx.name = fun_name
    tes = FactorTest(target_field=fun_name)

    tes.process_ind(pd.DataFrame(indx),data.ret_forward,simple=True)
    try:
        x = tes.get_ind_binned_ret_avg()
        if len(x) == 0:
            print('errrrrrrorlen(x) == 0\r\n')
            continue
    except :
        with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.width', 999):
            print(tes.ind_binned)
        print('errrrrrror\r\n')
        continue

    quality = check_quality(x)
    print(quality)
    # tes.plot(only_binned=True)
    # print('\r\n')



# # tes.ind_binned
# # tes.ind_ret_df
# x = tes.ind_ret_df.groupby(level=0, group_keys=False).apply(lambda x: pretreat.binning(x, deal_column=fun_name,box_count=10, inplace=True))
# if not x.group_label.isna().all():
#     with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.width', 999):
#         print(x)
# test.plot(only_binned=True)


alpha1
valid_right
alpha2
invalid
alpha3
invalid
alpha4
invalid
alpha5
valid_left
alpha6
valid_left
alpha7
sem_mid
alpha8
invalid
alpha9
invalid
alpha10
valid_right
alpha11
invalid
alpha12
valid_right
alpha13
valid_right
alpha14
valid_left
alpha15
invalid
alpha16
invalid
alpha17
valid_right
alpha18
invalid
alpha19
Empty DataFrame
Columns: [alpha19, ret_forward]
Index: []
errrrrrror

alpha20
invalid
alpha21
errrrrrrorlen(x) == 0

alpha22
invalid
alpha23
invalid
alpha24
invalid
alpha25
invalid
alpha26
valid_left
alpha27
errrrrrrorlen(x) == 0

alpha28
sem_mid
alpha29
valid_left
alpha30
valid_left
alpha31
invalid
alpha32
invalid
alpha33
valid_right
alpha34
valid_right
alpha35
invalid
alpha36
invalid
alpha37
invalid
alpha38
invalid
alpha39
Empty DataFrame
Columns: [alpha39, ret_forward]
Index: []
errrrrrror

alpha40
invalid
alpha41
valid_right
alpha42
valid_left
alpha43
valid_right
alpha44
invalid
alpha45
sem_mid
alpha46
sem_bilateral
alpha47
valid_left
alpha48
Empty DataFrame
Columns: [alp

In [593]:
ppp = inspect.signature(getattr(toys, 'alpha19')).parameters.keys()
aa = excute_for_multidates(data, lambda x: getattr(toys, 'alpha19')(*[x[param].copy() for param in ppp]) ,level=1)
aa.name = 'alpha19'
tt = FactorTest(target_field='alpha19')
tt.process_ind(pd.DataFrame(aa),data.ret_forward,simple=True)
tt.ind_ret_df

,,alpha19,ret_forward
date,code,,


In [624]:
ppp2 = inspect.signature(getattr(toys, 'alpha31')).parameters.keys()
aa2 = excute_for_multidates(data, lambda x: getattr(toys, 'alpha31')(*[x[param].copy() for param in ppp2]) ,level=1)
aa2.name = 'alpha31'
tt2 = FactorTest(target_field='alpha31')
tt2.process_ind(pd.DataFrame(aa2),data.ret_forward,simple=True)
tt2.get_ind_binned_ret_avg()
# tt2.ind_ret_df

group_label
1    -0.271883
2     0.706878
3     1.236217
4     0.433328
5    -1.290476
6     0.301921
7    -0.650514
8     0.390953
9    -0.930064
10   -1.057278
dtype: float64